# Context

One may filter different features in the neural signals. Here it is investigated which preprocessing steps are suitable in this respect.

# Imports

In [ ]:
from skimage import io
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, uniform_filter
import pickle

In [ ]:
import imageio
from pathlib import Path
from matplotlib.pyplot import show
from argparse import ArgumentParser

from pyoptflow import HornSchunck, getimgfiles
from pyoptflow.plots import compareGraphs

In [ ]:
from PIL import Image
import os
from scipy.signal import argrelextrema
from skimage import exposure

In [ ]:
import matplotlib
import matplotlib.animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [ ]:
np.array(np.clip([300],0,255), dtype=np.uint8)

### Import our custom utility methods

In [ ]:
import sys
%reload_ext autoreload
%autoreload 2
sys.path.append('..')

from utils.visualization_tools import *
import utils.visualization_tools
from utils.data_transformations import *
import utils.data_transformations
from utils.diverse import *
import utils.diverse

The following modules are available

In [ ]:
print_module_methods(utils.diverse)

In [ ]:
print_module_methods(utils.visualization_tools)

In [ ]:
print_module_methods(utils.data_transformations)

# Load data and inspect a frame of the raw data

In [ ]:
from pathlib import Path
source_folder = os.path.join(Path(os.getcwd()).parent, "source_data")

In [ ]:
frames = skimage.io.imread(os.path.join(source_folder,"runstart16_X1.tif"))

In [ ]:
frames = frames[:500,:,:]

# Preprocessing

Here I calculate the difference from pixelwise mean as well as a smoothed version that promised to increase the signal to noise ratio.

In [ ]:
mean = np.mean(frames,axis=0)#pixelwise mean

In [ ]:
difference = gaussian_filter(normalize(framewise_difference(frames, mean, bigdata=False)),1)
smooth = normalize(gaussian_filter(difference, 1))

In [ ]:
smoother = normalize(uniform_filter(smooth,[0,60,60]))#[1,10,10]))# [20,30,30]))

In [ ]:
details = heartbeat_filtered-smoother

In [ ]:
pca_in = details.reshape([details.shape[0],details.shape[1]*details.shape[2]])
pca = PCA(n_components=len(pca_in))
decompositions = pca.fit_transform(pca_in)
decompositions[:,3:] = 0
details = pca.inverse_transform(decompositions).reshape(details.shape)

In [ ]:
smooth = smoother+details

In [ ]:
heartbeat_filtered = remove_frequency_from_pixel_vectors(smooth,16,18)

In [ ]:
heartbeat_filtered = substract_pixel_min(heartbeat_filtered)

In [ ]:
%%capture
ani = show_video(heartbeat_filtered, n_frames=300)

In [ ]:
HTML(ani)

In [ ]:
from sklearn.decomposition import PCA
from scipy import signal


pca_in = difference.reshape([heartbeat_filtered.shape[0],heartbeat_filtered.shape[1]*heartbeat_filtered.shape[2]])
pca = PCA(n_components=len(pca_in))
decompositions = pca.fit_transform(pca_in)

In [ ]:
plt.plot(pca.explained_variance_ratio_.cumsum())

In [ ]:
decompositions[:,20:] = 0

In [ ]:
smooth = pca.inverse_transform(decompositions).reshape(heartbeat_filtered.shape)

In [ ]:
%%capture
fig, ax = plt.subplots(1, figsize=(10,10))

im = ax.imshow(smooth[0,:,:], vmin =.3, vmax=.5)#vmin=.25,vmax=.3)
startframe = 70
ani = matplotlib.animation.FuncAnimation(fig, lambda i: im.set_array(smooth[startframe+i]), frames=100).to_jshtml()

In [ ]:
HTML(ani)

# Horn and Schunck dense optical flow

In [ ]:
x_comp, y_comp = horn_schunck(smooth, 200)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], smooth[1])
start = 70
n_frames = 60

def animate(i):
    global start
    i += start
    print(".", end ="")    
    display_combined(x_comp[i]*50,y_comp[i]*50, smooth[i+1], fig=fig, ax=ax, quivstep = 3, vmin=.3,vmax=.5)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=n_frames)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
roi = sample_roi(details,0,100)

In [ ]:
x_comp, y_comp = horn_schunck(roi, len(roi)-1)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], details[1])
start = 0
frames = 10

def animate(i):
    i += start
    print(".", end ="")    
    display_combined(x_comp[i]/5,y_comp[i]/5, roi[i+1], fig=fig, ax=ax, scale=10, quivstep=1)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames)

In [ ]:
HTML(ani.to_jshtml())

# Conclusion

One can filter small scale motion patterns and largescale dynamics. The big size of the data represents a challange becuase of working memory restrictions when using NumPy methods directly. Custom methods can help to reduce the memory requirements. Developing scripts that run in a computational grid on computers with large memory capacities could also help.